In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State


# Configure the plotting routines
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
import re
import base64


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterCRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)

# df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Create an anchor for the image/logo, make the image an href to the website, www.snhu.edu,
    # and open the link in a new tab by setting a blank target
    html.A([
        html.Center(html.Img(src = 'data:image/png;base64,{}'.format(encoded_image.decode()),
            height = 250, width = 250))], href = 'https://www.snhu.edu', target = '_blank'),
    html.Center(html.B(html.H1('Michael Oelschlager SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # Create the radio buttons to act as a filter, and set the default on initial load to 'All'
    dcc.RadioItems(
        id = 'filter-type',
        options = [
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
        ],
        value = 'All'
    ),
    html.Hr(),
    dash_table.DataTable(
        id = 'datatable-id',
        columns = [
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
        # Set up the features for your interactive data table to make it user-friendly for your client
        editable = True,
        row_selectable = "single", # Row select funtionality
        selected_rows = [],
        filter_action = "native",  # Filter funtionality
        sort_action = "native",    # Sort funtionality
        page_action = "native",    # Pagination funtionality
        page_current = 0,            # Start page
        page_size = 12,            # Rows per page
    ),
    html.Br(),
    html.Hr(),
    # Set up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className = 'row',
        style = {'display': 'flex', 'justify-content': 'center'},
        children = [
            html.Div(
                id = 'graph-id',
                className = 'col s12 m6',
                ),
            html.Div(
                id = 'map-id',
                className = 'col s12 m6',
                )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will apply a filter to the data table when the user selects it
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #set up an if/else if/else block to respond to the radio buttons
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    elif filter_type == 'Water':
        # Use regex for pattern matching and build regex patterns for the different filters
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or':[ #Regex isn't allowed in an $in helper so use $or
                {"breed": {'$regex': newRegex}}, #pass the regex to the filter
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
        
    elif filter_type == 'Mountain':
        # Use regex for pattern matching and build regex patterns for the different filters
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old english.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': oldRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    
    elif filter_type == 'Disaster':
        # Use regex for pattern matching and build regex patterns for the different filters
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
    else:
        raise Exception('Unknown filter')
        
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    
    return(data, columns)

# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the pie chart with all of the data from all of the pages,
# giving the user a high-level viewpoint of all the relevant data in the database
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(virtualData):
    
    dffPieChart = pd.DataFrame.from_dict(virtualData)
    
    return [
        dcc.Graph(
            figure = px.pie(dffPieChart, names = 'breed',)
        )
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_selected_rows')])
def update_map(virtualRows):
    # Austin Texas is [30.75, -97.48]
    
    # Create the views
    if not virtualRows: # Build a default view if there are no selected lines
        markerArray = (30.75,-97.48) # Default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: # Build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) # Convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # Strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) # Strip out the long
        markerArray = (coordLat, coordLong) # Build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    # Return the map with a child marker set to the values found in markerArray
    # Map centers/moves to view the new marker instead of holding a fixed center
    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

app.run_server(debug=True)

TypeError: __init__() takes 1 positional argument but 3 were given